In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import kego.plotting

In [ ]:
signal_f = pd.read_parquet("~/Downloads/FGS1_signal_0_1010375142.parquet")
signal_c = pd.read_parquet("~/Downloads/AIRS-CH0_signal_0_1010375142.parquet")

In [ ]:
signal_f

In [ ]:
kego.plotting.plot_